In [2]:
import requests, pandas as pd
import glob
import os
import time
import re
import numpy as np

from dotenv import load_dotenv

In [4]:
# .env 파일 로드
load_dotenv()

# .env에서 API_KEY 불러오기
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY가 .env에서 로드되지 않았습니다. 확인필요!!")

In [6]:
RAW_DIR = "./data/raw/"
OUTPUT_DIR = "./data/working/"

# 디렉터리 보장
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. 2024년 6월 데이터 통합 파일 생성

### dataset) 서울시 따릉이 대여소별 대여/반납 승객수 정보

- https://data.seoul.go.kr/dataList/OA-21229/F/1/datasetView.do

In [10]:
output_csv = os.path.join(OUTPUT_DIR, "tpss_bcycl_od_statnhm_202406_통합.csv")

# 통합 파일 존재 여부 확인
if os.path.exists(output_csv):
    print(f"이미 통합 파일이 존재합니다: {output_csv}")
else:
    # 대상 파일 목록
    files = sorted(glob.glob(os.path.join(RAW_DIR, "tpss_bcycl_od_statnhm_202406*.csv")))
    print(f"파일 수: {len(files)}")

    if not files:
        raise FileNotFoundError("원본 CSV가 없습니다.")

    # 모든 파일 읽어서 하나의 DF로
    df_list = []
    for f in files:
        for enc in ("cp949", "euc-kr", "utf-8-sig"):
            try:
                df = pd.read_csv(f, encoding=enc, low_memory=False)
                df_list.append(df)
                break
            except UnicodeDecodeError:
                continue
        else:
            raise RuntimeError(f"인코딩 감지 실패: {f}")

    df_all = pd.concat(df_list, ignore_index=True)

    # CSV 저장
    df_all.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"저장 완료: {output_csv}")

파일 수: 30
저장 완료: ./data/working/tpss_bcycl_od_statnhm_202406_통합.csv


In [12]:
data1 = pd.read_csv("./data/working/tpss_bcycl_od_statnhm_202406_통합.csv")

In [13]:
data1.shape

(8831528, 10)

In [16]:
data1.head()

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
0,20240601,출발시간,0,ST-1002,목1동_004_1,ST-1017,목5동_059_1,1,8.0,870.0
1,20240601,출발시간,0,ST-1015,목5동_001_1,ST-997,목4동_021_1,1,10.0,1552.0
2,20240601,출발시간,0,ST-1036,역촌동_001_1,ST-1035,불광2동_021_1,1,42.0,4980.0
3,20240601,출발시간,0,ST-1045,성내2동_007_1,ST-1580,오륜동_001_3,1,8.0,1923.0
4,20240601,출발시간,0,ST-1047,성내1동_023_1,ST-488,암사1동_044_1,1,18.0,3530.0


In [18]:
data1.dtypes

기준_날짜          int64
집계_기준         object
기준_시간대         int64
시작_대여소_ID     object
시작_대여소명       object
종료_대여소_ID     object
종료_대여소명       object
전체_건수          int64
전체_이용_분      float64
전체_이용_거리     float64
dtype: object

In [20]:
data1.columns

Index(['기준_날짜', '집계_기준', '기준_시간대', '시작_대여소_ID', '시작_대여소명', '종료_대여소_ID',
       '종료_대여소명', '전체_건수', '전체_이용_분', '전체_이용_거리'],
      dtype='object')

# 2. 공공자전거 대여소 정보 가져오기

### dataset) 서울시 공공자전거 대여소 정보

- https://data.seoul.go.kr/dataList/OA-13252/F/1/datasetView.do

### csv 파일 사용 (6월 기준)

In [35]:
file = "./data/raw/공공자전거_대여소_정보(24.6월_기준).xlsx"
output = "./data/working/공공자전거_대여소_정보_CSV.csv"

# output 파일 존재 여부 확인
if os.path.exists(output):
    print("이미 파일이 존재합니다:", output)
else:
    data_excel = pd.read_excel(file, sheet_name="대여소현황", header=None)

    # 6행부터 실제 데이터
    data_excel = data_excel.iloc[5:].copy()

    # 커스텀 헤더
    custom_header = ["대여소번호", "보관소(대여소)명", "자치구", "상세주소", 
                     "위도", "경도", "설치시기", "LCD거치대수", "QR거치대수", "운영방식"]
    data_excel.columns = custom_header

    # '대여소번호' 문자열 처리 (앞에 0 보존)
    data_excel["대여소번호"] = data_excel["대여소번호"].astype(str).str.zfill(5)

    # csv 저장
    os.makedirs(os.path.dirname(output), exist_ok=True)
    data_excel.to_csv(output, index=False, encoding="utf-8-sig")
    print("저장 완료:", output)

이미 파일이 존재합니다: ./data/working/공공자전거_대여소_정보_CSV.csv


In [37]:
data_csv = pd.read_csv("./data/working/공공자전거_대여소_정보_CSV.csv")

In [39]:
data_csv.head()

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD거치대수,QR거치대수,운영방식
0,301,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,2015-10-07 12:03:46,20.0,20.0,QR
1,302,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.974060,2015-10-07 12:04:22,12.0,12.0,QR
2,303,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.571770,126.974663,2015-10-07 00:00:00,8.0,8.0,QR
3,305,종로구청 옆,종로구,서울특별시 종로구 세종로 84-1,37.572559,126.978333,2015-01-07 00:00:00,16.0,16.0,QR
4,307,서울역사박물관 앞,종로구,서울특별시 종로구 새문안로 55 서울역사박물관 앞,37.570000,126.971100,2015-10-07 12:09:09,11.0,11.0,QR


In [31]:
data_csv.shape

(2763, 10)

In [33]:
data_csv.columns

Index(['대여소번호', '보관소(대여소)명', '자치구', '상세주소', '위도', '경도', '설치시기', 'LCD거치대수',
       'QR거치대수', '운영방식'],
      dtype='object')

### api 사용

In [41]:
SERVICE = "tbCycleStationInfo"
BASE = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE}"
PAGE_SIZE = 1000
SLEEP_SEC = 0.2

output_path = "./data/working/공공자전거_대여소_정보_API.csv"

if os.path.exists(output_path):
    print("이미 파일이 존재합니다:", output_path)
else:
    all_rows = []
    start = 1

    while True:
        end = start + PAGE_SIZE - 1
        url = f"{BASE}/{start}/{end}/"
        
        try:
            res = requests.get(url, timeout=30)
            res.raise_for_status()
            js = res.json()
        except Exception as e:
            print(f"[ERROR] {start}-{end} 구간 요청 실패: {e}")
            break
    
        # 응답 파싱
        root = js.get(SERVICE) or (js.get(list(js.keys())[0]) if js else None)
        rows = (root or {}).get("row", [])
    
        if not rows:
            print(f"데이터 없음: {start}-{end} 구간에서 종료")
            break
    
        all_rows.extend(rows)
    
        # 마지막 페이지(1000건 미만)면 종료
        if len(rows) < PAGE_SIZE:
            break
    
        # 다음 구간으로
        start += PAGE_SIZE
        time.sleep(SLEEP_SEC)

    # DataFrame 변환
    data_api = pd.DataFrame(all_rows)
    
    # 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    data_api.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"저장 완료 : {output_path}")

저장 완료 : ./data/working/공공자전거_대여소_정보_API.csv


In [43]:
data_api = pd.read_csv(output_path, encoding="utf-8-sig")

In [45]:
data_api.head()

,STA_LOC,RENT_ID,RENT_NO,RENT_NM,RENT_ID_NM,HOLD_NUM,STA_ADD1,STA_ADD2,STA_LAT,STA_LONG,START_INDEX,END_INDEX,RNUM
0,마포구,ST-10,108,서교동 사거리,108. 서교동 사거리,12.0,서울특별시 마포구 양화로 93,427,37.552746,126.918617,0,0,1
1,광진구,ST-100,503,더샵스타시티 C동 앞,503. 더샵스타시티 C동 앞,15.0,서울특별시 광진구 아차산로 262,더샵스타시티 C동 앞,37.536667,127.073593,0,0,2
2,양천구,ST-1000,729,서부식자재마트 건너편,729. 서부식자재마트 건너편,10.0,서울특별시 양천구 신정동 236,서부식자재마트 건너편,37.510380,126.866798,0,0,3
3,양천구,ST-1002,731,서울시 도로환경관리센터,731. 서울시 도로환경관리센터,10.0,서울특별시 양천구 목동동로 316-6,서울시 도로환경관리센터,37.529900,126.876541,0,0,4
4,양천구,ST-1003,732,신월중학교,732. 신월중학교,10.0,서울특별시 양천구 화곡로 59,신월동 이마트,37.539551,126.828300,0,0,5


In [47]:
data_api.shape

(3180, 13)

In [49]:
data_api.dtypes

STA_LOC         object
RENT_ID         object
RENT_NO          int64
RENT_NM         object
RENT_ID_NM      object
HOLD_NUM       float64
STA_ADD1        object
STA_ADD2        object
STA_LAT        float64
STA_LONG       float64
START_INDEX      int64
END_INDEX        int64
RNUM             int64
dtype: object

In [51]:
data_api.columns

Index(['STA_LOC', 'RENT_ID', 'RENT_NO', 'RENT_NM', 'RENT_ID_NM', 'HOLD_NUM',
       'STA_ADD1', 'STA_ADD2', 'STA_LAT', 'STA_LONG', 'START_INDEX',
       'END_INDEX', 'RNUM'],
      dtype='object')

# 3. 6월 데이터 통합본 + 자치구 매핑

In [54]:
output_path = "./data/working/tpss_bcycl_od_statnhm_202406_통합_w_자치구.csv"

seoul_gu = data_api.set_index("RENT_ID")["STA_LOC"]

# data1에 컬럼 생성
if "시작_대여소_자치구" not in data1.columns:
    data1["시작_대여소_자치구"] = data1["시작_대여소_ID"].map(seoul_gu)

if "종료_대여소_자치구" not in data1.columns:
    data1["종료_대여소_자치구"] = data1["종료_대여소_ID"].map(seoul_gu)

# 컬럼 순서
new_cols = [
    "기준_날짜", "집계_기준", "기준_시간대",
    "시작_대여소_ID", "시작_대여소명", "시작_대여소_자치구",
    "종료_대여소_ID", "종료_대여소명", "종료_대여소_자치구",
    "전체_건수", "전체_이용_분", "전체_이용_거리"
]
data1 = data1[new_cols]

if os.path.exists(output_path):
    print("이미 존재합니다:", output_path)
else:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    data1.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"저장 완료: {output_path}")

저장 완료: ./data/working/tpss_bcycl_od_statnhm_202406_통합_w_자치구.csv


In [56]:
data1_merged = pd.read_csv(output_path)

In [57]:
data1_merged.shape

(8831528, 12)

In [58]:
data1_merged.head()

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,시작_대여소_자치구,종료_대여소_ID,종료_대여소명,종료_대여소_자치구,전체_건수,전체_이용_분,전체_이용_거리
0,20240601,출발시간,0,ST-1002,목1동_004_1,양천구,ST-1017,목5동_059_1,양천구,1,8.0,870.0
1,20240601,출발시간,0,ST-1015,목5동_001_1,양천구,ST-997,목4동_021_1,양천구,1,10.0,1552.0
2,20240601,출발시간,0,ST-1036,역촌동_001_1,은평구,ST-1035,불광2동_021_1,은평구,1,42.0,4980.0
3,20240601,출발시간,0,ST-1045,성내2동_007_1,강동구,ST-1580,오륜동_001_3,송파구,1,8.0,1923.0
4,20240601,출발시간,0,ST-1047,성내1동_023_1,강동구,ST-488,암사1동_044_1,강동구,1,18.0,3530.0


In [68]:
# null 값이 있는 행만 필터링
null_rows = data1[
    data1["시작_대여소_자치구"].isna() | data1["종료_대여소_자치구"].isna()
]

print("null 값 있는 행 :", len(null_rows))
null_rows.head()

# # CSV 저장
# output_path = os.path.join(OUTPUT_DIR, "tpss_bcycl_od_statnhm_202406_통합_w_자치구null.csv")
# null_rows.to_csv(output_path, index=False, encoding="utf-8-sig")
# print(f"null행 저장 완료: {output_path}")

null 값 있는 행 : 31838


,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,시작_대여소_자치구,종료_대여소_ID,종료_대여소명,종료_대여소_자치구,전체_건수,전체_이용_분,전체_이용_거리
132,20240601,출발시간,0,ST-2308,화양동_041_2,광진구,X,NaN,NaN,1,114.0,605.0
805,20240601,출발시간,10,ST-1236,신도림동_062_1,구로구,X,NaN,NaN,1,60.0,3026.0
832,20240601,출발시간,10,ST-1353,영등포본동_044_1,영등포구,X,NaN,NaN,1,20.0,1976.0
943,20240601,출발시간,10,ST-2502,용신동_003_1,동대문구,X,NaN,NaN,1,60.0,7054.0
1103,20240601,출발시간,10,ST-910,월곡2동_015_1,성북구,X,NaN,NaN,1,4.0,533.0


# 4. 시간대별, 자치구별 그룹화

In [101]:
# 원본 보존
df = data1_merged.copy()

df["기준_월"] = "202406"

# 출발시간/도착시
df["집계_기준"] = (
    df["집계_기준"]
    .astype(str)
      .str.replace("_", "", regex=False)
      .str.strip()
)

df = df[df["집계_기준"].isin(["출발시간", "도착시간"])].copy()

# 숫자형
df["전체_건수"]   = pd.to_numeric(df["전체_건수"], errors="coerce").fillna(0).astype(int)
df["기준_시간대"] = pd.to_numeric(df["기준_시간대"], errors="coerce").fillna(0).astype(int)

# 기준_시간대_1시간 (0~23)
df["기준_시간대_1시간"] = (df["기준_시간대"] // 12).astype(int).clip(0, 23)

# 출발 > 시작_대여소_자치구, 도착 > 종료_대여소_자치구
df["자치구"] = np.where(
    df["집계_기준"] == "출발시간",
    df["시작_대여소_자치구"],
    df["종료_대여소_자치구"]
)

# 자치구 × 1시간 × 집계기준 그룹화
grouped = (
    df.groupby(["기준_월", "기준_시간대_1시간", "자치구", "집계_기준"], as_index=False)["전체_건수"]
      .sum()
      .sort_values(["기준_월", "기준_시간대_1시간", "자치구", "집계_기준"])
)

# Pivot
pivot_result = (
    grouped.pivot_table(
        index=["기준_월", "기준_시간대_1시간", "자치구"],
        columns="집계_기준",
        values="전체_건수",
        fill_value=0
    )
    .reset_index()
)

pivot_result.columns.name = None

# 컬럼명 변경
pivot_result = pivot_result.rename(
    columns={"출발시간": "대여량", "도착시간": "반납량"}
)

# 지표 계산
if {"대여량", "반납량"}.issubset(pivot_result.columns):
    pivot_result["순이동(대여-반납)"] = pivot_result["대여량"] - pivot_result["반납량"]
    pivot_result["총처리량(대여+반납)"] = pivot_result["대여량"] + pivot_result["반납량"]

In [102]:
grouped.shape

(850, 5)

In [103]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   기준_월        850 non-null    object
 1   기준_시간대_1시간  850 non-null    int32 
 2   자치구         850 non-null    object
 3   집계_기준       850 non-null    object
 4   전체_건수       850 non-null    int32 
dtypes: int32(2), object(3)
memory usage: 26.7+ KB


In [104]:
grouped.head()

,기준_월,기준_시간대_1시간,자치구,집계_기준,전체_건수
0,202406,0,강남구,도착시간,293
1,202406,0,강남구,출발시간,969
2,202406,0,강동구,도착시간,524
3,202406,0,강동구,출발시간,1565
4,202406,0,강북구,도착시간,158


In [105]:
grouped.tail()

,기준_월,기준_시간대_1시간,자치구,집계_기준,전체_건수
845,202406,23,종로구,출발시간,132458
846,202406,23,중구,도착시간,92762
847,202406,23,중구,출발시간,96916
848,202406,23,중랑구,도착시간,129033
849,202406,23,중랑구,출발시간,126721


In [111]:
pivot_result.head()

,기준_월,기준_시간대_1시간,자치구,반납량,대여량,순이동(대여-반납),총처리량(대여+반납)
0,202406,0,강남구,293.0,969.0,676.0,1262.0
1,202406,0,강동구,524.0,1565.0,1041.0,2089.0
2,202406,0,강북구,158.0,635.0,477.0,793.0
3,202406,0,강서구,1052.0,2891.0,1839.0,3943.0
4,202406,0,관악구,255.0,896.0,641.0,1151.0


In [113]:
pivot_result.tail()

,기준_월,기준_시간대_1시간,자치구,반납량,대여량,순이동(대여-반납),총처리량(대여+반납)
420,202406,23,용산구,88798.0,88823.0,25.0,177621.0
421,202406,23,은평구,111659.0,108972.0,-2687.0,220631.0
422,202406,23,종로구,127624.0,132458.0,4834.0,260082.0
423,202406,23,중구,92762.0,96916.0,4154.0,189678.0
424,202406,23,중랑구,129033.0,126721.0,-2312.0,255754.0


In [115]:
pivot_result.shape

(425, 7)